<div><img style="float: right; width: 120px; vertical-align:middle" src="https://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/EU_Informatica/ETSI%20SIST_INFORM_COLOR.png" alt="ETSISI logo" />


# Image noise suppression with denoising autoencoders<a id="top"></a>

<i><small>Autor: Alberto Díaz Álvarez<br>Última actualización: 2023-04-27</small></i></div>
                                                  

***

## Introduction

Denoising autoencoders (first presented on [Extracting and Composing Robust Features with Denoising Autoencoders](https://www.cs.toronto.edu/~larocheh/publications/icml-2008-denoising-autoencoders.pdf)) are a type of neural network whosepurpose is to remove noise from input data so that it can be used more effectively in downstream tasks. Noise can be introduced into data in many ways, such as in images where noise can be caused by factors like lighting conditions, sensor errors or compression artifacts. Removing this noise can lead to better performance in tasks such as image recognition or classification.

| <img src="https://production-media.paperswithcode.com/methods/Denoising-Autoencoder_qm5AOQM.png" alt="Denoising autoencoder" width="50%"> | 
|:--:| 
| *A denoising autoencoder is purposely trained with noise-corrupted data to learn how to remove it effectively. Source: Kumar, V., Nandi, G. C., & Kala, R. (2014, August). [Static hand gesture recognition using stacked denoising sparse autoencoders](https://ieeexplore.ieee.org/document/6897155). In 2014 7th IC3 (pp. 99-104). IEEE.]( (last visited April 24, 2023).* |

Denoising autoencoders work by training a neural network to learn a compressed representation of the input data, which is then used to reconstruct the original data but without the noise. This process involves adding artificial noise to the input data and training the neural network to reconstruct the original data without the noise. The network is trained to identify patterns in the input data that are consistent across the different examples of the same class and to ignore the patterns that are not consistent.

## Goals

Our goals are to understand what a denoising autoencoder is, build a denoising autoencoder using Keras, train the denoising autoencoder on the Fashion MNIST dataset and generate denoised images using the trained autoencoder.

## Libraries and configuration

Next we will import the libraries that will be used throughout the notebook.

In [ ]:
import math
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

We will also configure some parameters to adapt the graphical presentation.

In [ ]:
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams["axes.grid"] = False
plt.rcParams.update({'figure.figsize': (20, 6),'figure.dpi': 64})

***

## Dataset

We'll use the Fashion MNIST datasetin the same way we did on the previous notebook

In [ ]:
(x_train, _), (x_test, _) = tf.keras.datasets.fashion_mnist.load_data()
x_train, x_test = x_train / 255, x_test / 255

print(f'Training shape: {x_train.shape} input')
print(f'Test shape:     {x_test.shape} input')

## A noise layer

Here is a possible implementation of a custom layer that adds Gaussian noise to its input to output the same input but messy.

In [ ]:
class GaussianNoiseLayer(tf.keras.layers.Layer):
    def __init__(self, μ=.0, σ=1., *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.μ = μ
        self.σ = σ

    def call(self, x, training=False):
        if training:
            x += tf.random.normal(x.shape, mean=self.μ, stddev=self.σ)

        return x

Ok, in Keras we already have an implementation for adding gaussian noise (yet always centered on 0, so our layer is more generic), the `tf.keras.layers.GaussianNoise` layer, although this implementation allows us to see an argument that we had not seen before. If we look at it, the layer **only acts when we are training**, and does nothing if the mode is "not training" (i.e. inferring). Ya que la hemos implementado, vamos a usarla para añadir ruido en lugar de la existente en la librería.

## Denoising autoencoder implementation

Now, let's go to the implementation of the denoising autoencoder, which we will see that it is very similar to the autoencoders we have seen so far. We will use as a baseline the vanilla autoencoder, and we will build our denoising autoencoder from it.

In [ ]:
class DenoisingAutoencoder(tf.keras.models.Model):
    """Represents a simple denoiseing autoencoder"""
    def __init__(self, input_dim, latent_dim, noise_mean=.0, noise_stddev=1.):
        super().__init__()

        flatten_dim = None
        if isinstance(input_dim, (list, tuple)):
            flatten_dim = math.prod(input_dim)
        elif isinstance(input_dim, int):
            flatten_dim = input_dim
            input_dim = (input_dim,)
        else:
            raise ValueError('Argument input_dim must be a tuple or an int')
        
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Flatten(),
            GaussianNoiseLayer(μ=noise_mean, σ=noise_stddev),
            tf.keras.layers.Dense(latent_dim, activation=tf.keras.layers.LeakyReLU()),
        ])
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Dense(flatten_dim, activation='sigmoid'),
            tf.keras.layers.Reshape(input_dim)
        ])

    def call(self, x):
        return self.decoder(self.encoder(x))

We will create a new denoising autoencoder with a considerable some of noise.

In [ ]:
dae = DenoisingAutoencoder(input_dim=(28, 28), latent_dim=256, noise_mean=.0, noise_stddev=2.)
dae.compile(loss='binary_crossentropy', optimizer='adam')

And we will train him to see how he behaves.

In [ ]:
history = dae.fit(x_train, x_train, epochs=100)

How good has the training been?

In [ ]:
pd.DataFrame(history.history).plot()
plt.xlabel('Epoch num.')
plt.show()

Well, good in a way, although it could have been much better. Let's look at some coding/decoding examples from the training set.

In [ ]:
n = 4
images = np.array(random.sample(list(x_train), n))

encoded = dae.encoder(images).numpy()
decoded = dae.decoder(encoded).numpy()
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(images[i])
    plt.title('Original')

    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded[i])
    plt.title('Reconstructed')

And with data you have never seen?

In [ ]:
images = np.array(random.sample(list(x_test), n))
encoded = dae.encoder(images).numpy()
decoded = dae.decoder(encoded).numpy()
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(images[i])
    plt.title('Original')

    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded[i])
    plt.title('Reconstructed')

He doesn't seem to be doing too badly. Let's see now how it behaves by adding a lot of noise to the images during the training..

In [ ]:
dae = DenoisingAutoencoder(input_dim=(28, 28), latent_dim=192, noise_mean=.0, noise_stddev=6.)
dae.compile(loss='binary_crossentropy', optimizer='adam')

Let us now train our model with the training set.

In [ ]:
history = dae.fit(x_train, x_train, epochs=100)

Let's see how the training has evolved:

In [ ]:
pd.DataFrame(history.history).plot()
plt.xlabel('Epoch num.')
plt.show()

The training doesn't seem to be very good. Let's see how some examples of the training set are encoded and decoded.

In [ ]:
n = 4
images = np.array(random.sample(list(x_train), n))

encoded = dae.encoder(images).numpy()
decoded = dae.decoder(encoded).numpy()
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(images[i])
    plt.title('Original')

    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded[i])
    plt.title('Reconstructed')

Now let's see with data you have never seen before

In [ ]:
images = np.array(random.sample(list(x_test), n))
encoded = dae.encoder(images).numpy()
decoded = dae.decoder(encoded).numpy()
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(images[i])
    plt.title('Original')

    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded[i])
    plt.title('Reconstructed')

## Conclusion

Denoising autoencoders have a wide range of applications in computer vision, including image denoising, inpainting, and super-resolution (although we have only seen its application here on a very small toy data set). They can also be applied to other types of data, such as audio and text.

One advantage of denoising autoencoders is that they are self-supervised, which means that they don't require labeled data to train. This can be particularly useful in cases where labeled data is scarce or expensive to obtain.

However, there are some limitations to denoising autoencoders. They may not be effective in cases where the noise is too severe or where the noise pattern is complex and difficult to model. Additionally, denoising autoencoders are not always able to preserve fine details in the input data, especially if the noise is highly correlated with these details.

In summary, denoising autoencoders are a powerful tool for removing noise from input data and can be applied to a wide range of applications in computer vision and other fields. However, their effectiveness depends on the nature and severity of the noise and the complexity of the noise pattern, as well as the specific details of the data being processed.

***

<div><img style="float: right; width: 120px; vertical-align:top" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" alt="Creative Commons by-nc-sa logo" />

[Volver al inicio](#top)

</div>